In [24]:
# Guillaume Rozier - 2020 - MIT License
# This script will automatically tweet new data and graphes on the account @covidtracker_fr

# importing the module 

import france_data_management as data
import math
from datetime import datetime
import locale
import tweepy

import secrets as s

def tweet():
    locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')

    """
    Secrets :
        consumer_key ="xxxxxxxxxxxxxxxx"
        consumer_secret ="xxxxxxxxxxxxxxxx"
        access_token ="xxxxxxxxxxxxxxxx"
        access_token_secret ="xxxxxxxxxxxxxxxx"
    """

    _, _, dates, df_new, _, _, _, df_incid = data.import_data()
    df_new_france = df_new.groupby(["jour"]).sum().reset_index()
    df_incid_france = df_incid.groupby(["jour"]).sum().reset_index()

    hosp = df_new_france.iloc[len(df_new_france)-1]['incid_hosp']
    deaths = df_new_france.iloc[len(df_new_france)-1]['incid_dc']
    cases = df_incid_france.iloc[len(df_incid_france)-1]['p']

    # authentication 
    auth = tweepy.OAuthHandler(s.consumer_key, s.consumer_secret) 
    auth.set_access_token(s.access_token, s.access_token_secret) 

    api = tweepy.API(auth) 

    date = datetime.strptime(dates[-1], '%Y-%m-%d').strftime('%d %B')
    date_incid = datetime.strptime(sorted(list(dict.fromkeys(list(df_incid_france['jour'].values))))[-1], '%Y-%m-%d').strftime('%d %B')
    tweet ="#Covid19 en #France - Données du jour :\n• {} nouveaux cas positifs ({})\n• {} personnes décédées en milieu hosp. ({})\n• {} admissions à l'hôpital ({})\n➡️ Plus d'infos : covidtracker.fr/covidtracker-france".format(cases, date_incid, deaths, date, hosp, date) # toDo 
    image_path ="images/charts/france/var_journ_lines.jpeg"

    # to attach the media file 
    status = api.update_with_media(image_path, tweet)
    #print(tweet)

10it [00:02,  3.46it/s]                      


In [18]:
#import france_data_management as data
#df, df_confirmed, dates, df_new, df_tests, df_deconf, df_sursaud, df_incid = data.import_data()


106